In [2]:
from logging import exception
from music21 import converter, stream
import csv
import configparser
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], '..'))

try:
    DATA_PATH = '../data'
    DATASET_PATH = DATA_PATH + '/KYDataset'
    GT_PATH = '../data/ground_truth'
    RESULT_PATH = '../result'
except KeyError:
    print("failed to read config.ini, or invalid index specified")
    raise SystemExit


class Piece:
    """Container of a music21.stream object with custom data extration methods and enhanced abstraction.
    """

    def __init__(self, filename):
        """
        :param filename: filename of the music piece (with or without extension)
        :type filename: str
        """
        self.name = filename[:-4] if filename.endswith('.mxl') else filename
        self.score = converter.parse(os.path.join(DATASET_PATH, self.name+'.mxl'))
        self.length = self.score.duration.quarterLength

        self.chordified = self.score.chordify()
        self.flattened = self.score.flatten()

    def __iter__(self):
        """
        :return: Iterator of current stream object
        :rtype: StreamIterator
        """
        return stream.iterator.StreamIterator(self.score)

    def __get_elements_in_measures(self, measures, type):
        res = []
        for measure in measures:
            for el in measure.getElementsByClass(type):
                res.append(el)
        return res

    def get_key_signatures(self, custom_stream=None):
        """Search key signatures for all parts of the score.

        :rtype: list of keySignature objects
        """
        target_stream = self.score if not custom_stream else custom_stream
        return list(target_stream.recurse().getElementsByClass("KeySignature"))

    def get_elements_by_offset(self, filter=None):
        """Obtain all/specified elements in flatterned stream, ordered by offset

        :param filter: filter type based on m21 classes, defaults to None
        :type filter: str/m21 class, optional
        :return: dictionary of offsets and lists of m21 objects in {<offset>: [<element>]}
        :rtype: dict
        """
        o_iter = stream.iterator.OffsetIterator(self.flattened)
        if filter:
            o_iter = o_iter.getElementsByClass(filter)
        return {x[0].offset: x for x in o_iter}

    def get_measures(self, custom_stream=None):
        """Obtain all measures of the score. Default to origin stream.

        :param custom_stream: specify stream (compactible with chordified stream), defaults to None
        :type custom_stream: music21.stream.Score, optional
        :return: (default) list of lists of measures
                 (custom) list of measures
        :rtype: list
        """
        if not custom_stream:
            parts_iter = self.score.getElementsByClass("PartStaff")
            return [list(part.getElementsByClass('Measure')) for part in parts_iter]
        else:
            return list(custom_stream.recurse().getElementsByClass('Measure'))

    def get_notes(self, custom_stream=None):
        """Obtain all notes of the score.

        :param custom_stream: specify stream, defaults to None
        :type custom_stream: music21.stream.Score, optional
        :return: (default) list of lists of notes
                 (custom) list of notes
        :rtype: list
        """
        if not custom_stream:
            parts = self.get_measures()
            return [self.__get_elements_in_measures(measures, 'Note') for measures in parts]
        else:
            return list(custom_stream.recurse().getElementsByClass('Note'))

    def get_ground_truth(self, type='chord'):
        """Traverse labelled music stream to generates text-based ground truth data for evaluation.

        :param type: 'chord' or 'key' segments, defaults to 'chord'
        :type type: str, optional
        :return: list of tuples of float, string, char, string in (<offset>, <tonic>, <key>, <chord>);
                 or list of tuples of float, string, char in (<offset>, <tonic>, <key>)
        :rtype: list
        """
        res = []
        notes = self.get_elements_by_offset(filter="Note")
        # if type == 'chord':
        # chord_symbols = self.get_elements_by_offset(filter="ChordSymbol")
        # for offset, el in chord_symbols.items():
        #     chord = el[0]
        #     cs = f'{chord.figure}({chord.chordKindStr})'
        #     res.append((offset, cs))

        if type == 'chord':
            for offset, el in notes.items():
                note = el[0]
                if note.lyric:
                    try:
                        if '(' in note.lyric:
                            scale = note.lyric.split('(')[0]
                            chord = note.lyric.split('(')[1][:-1]
                            res.append((note.offset, scale[:-1], scale[-1], chord))
                        else:
                            res.append(
                                (note.offset, scale[:-1], scale[-1], note.lyric))
                    except Exception:
                        print(note.lyric)
        elif type == 'key':
            for offset, el in notes.items():
                note = el[0]
                if note.lyric and '(' in note.lyric:
                    scale = note.lyric.split('(')[0]
                    res.append((note.offset, scale[:-1], scale[-1]))
        else:
            raise ValueError('Type must be either chord or key')

        return res

    def export(self, path=RESULT_PATH, filename=None, type='mxl'):
        """Output music stream as mxl file (by default).

        :param path: path to write on, defaults to OUTPUT_PATH
        :type path: str, optional
        :param type: output format, defaults to 'mxl'
        :type type: str, optional
        """
        name = self.filename if not filename else filename
        filepath = os.path.join(path, name)
        self.score.write(type, fp=filepath)

    def _export_ground_truth(self):
        """Single use function for generating ground truth csv files
        """
        try:
            gt = self.get_ground_truth()

            path = os.path.join(GT_PATH, self.name + ".csv")
            file = open(path, "w", newline="")
            writer = csv.writer(file)
            writer.writerow(("offset", "tonic", "key", "chord"))
            for segment in gt:
                writer.writerow(segment)
            file.close()

            gt = self.get_ground_truth(type='key')

            path = os.path.join(GT_PATH + "_key", self.name + ".csv")
            file = open(path, "w", newline="")
            writer = csv.writer(file)
            writer.writerow(("offset", "tonic", "key"))
            for segment in gt:
                writer.writerow(segment)
            file.close()

        except Exception:
            print(self.name)
            print(gt)

    def show(self):
        self.score.show()


In [32]:
p = Piece('Chopin_F._Nocturne_in_C-Sharp_Minor,_Op.posth._No.20.mxl')
for off, el in p.get_elements_by_offset().items():
    for n in el:
        print(off, el, n.lyric, end=' ')
    print()

AttributeError: 'TextBox' object has no attribute 'lyric'

In [22]:
p.score.show('text')

{0.0} <music21.text.TextBox 'Nocturne N...'>
{0.0} <music21.text.TextBox 'Opus Posth...'>
{0.0} <music21.text.TextBox 'Frédéric F...'>
{0.0} <music21.metadata.Metadata object at 0x7f80a9bc56d0>
{0.0} <music21.stream.PartStaff P1-Staff1>
    {0.0} <music21.instrument.Piano 'P1: Piano: Piano'>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.expressions.TextExpression 'Lento con ...'>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.KeySignature of 4 sharps>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.dynamics.Dynamic p>
        {0.0} <music21.chord.Chord E4 G#4 C#5>
        {1.0} <music21.note.Rest eighth>
        {1.5} <music21.chord.Chord C#4 G#4 B4>
        {2.0} <music21.chord.Chord C#4 F#4 A4>
        {3.0} <music21.note.Rest eighth>
        {3.5} <music21.chord.Chord C#4 E4 G#4>
    {0.0} <music21.spanner.Slur <music21.chord.Chord C#4 G#4 B4><music21.chord.Chord C#4 F#4 A

In [63]:
print(p.length)
for el in p.score.recurse().notes:
    print(el.offset, el.lyric)

59581/240
0.0 C#m(I)
1.5 None
2.0 IV
3.5 I
0.0 GerVI
1.0 V+
0.0 I
1.5 None
2.0 IV
3.5 I
0.0 GerVI
1.0 V+
0.0 I
2.0 II
2.0 None
2.125 None
2.25 None
2.375 None
2.5 None
2.625 None
2.75 None
2.8125 None
2.875 None
2.9375 None
3.0 None
3.0625 None
3.125 None
3.1875 None
3.25 None
3.3125 None
3.375 None
3.4375 None
3.5 None
3.5625 None
3.625 None
3.6875 None
3.75 None
91/24 None
23/6 None
3.875 None
3.9375 None
0.0 I
2.0 None
0.0 I+
2.0 None
2.75 None
3.0 None
10/3 None
11/3 None
0.0 II
0.0 V+
2.0 None
0.0 I
2.0 IV
3.0 None
0.0 None
0.0 None
0.0 None
3.0 None
3.5 None
0.0 V+
0.125 None
0.25 None
0.375 None
0.5 None
0.625 None
0.75 None
0.875 None
1.0 None
1.125 None
1.25 None
1.375 None
1.5 None
1.625 None
1.75 None
1.8125 None
1.875 None
1.9375 None
2.0 None
2.0625 None
2.125 None
2.1875 None
2.25 None
2.3125 None
2.375 None
2.4375 None
2.5 None
2.5625 None
2.625 None
2.6875 None
2.75 None
2.8125 None
2.875 None
2.9375 None
3.0 None
3.0625 None
3.125 None
3.1875 None
3.25 None
3.3125 None

In [65]:
for k, v in p.get_elements_by_offset(filter='Chord').items():
    for el in v:
        print(el.lyric, end=' ')
    print()

C#m(I) 
None 
IV 
I 
GerVI 
V+ 
I 
None 
IV 
I 
GerVI 
V+ 
None 
None 
None 
I 
None 
None V 
None 
None 
None 
I 
C#m(V+) 
None 
None 
I 
None 
None V+7 
None 
None 
None 
I 


In [68]:
for el in p.score.recurse().getElementsByClass(['Note', 'Chord']):
    print(el)

<music21.chord.Chord E4 G#4 C#5>
<music21.chord.Chord C#4 G#4 B4>
<music21.chord.Chord C#4 F#4 A4>
<music21.chord.Chord C#4 E4 G#4>
<music21.chord.Chord C#4 E4 F##4>
<music21.chord.Chord B#3 D#4 G#4>
<music21.chord.Chord E4 G#4 C#5>
<music21.chord.Chord C#4 G#4 B4>
<music21.chord.Chord C#4 F#4 A4>
<music21.chord.Chord C#4 E4 G#4>
<music21.chord.Chord C#4 E4 F##4>
<music21.chord.Chord B#3 D#4 G#4>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note G#>
<m

In [57]:
dir(p.score.recurse())

['_DOC_ATTR',
 '_DOC_ORDER',
 '__annotations__',
 '__bool__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_classListFullyQualifiedCacheDict',
 '_classSetCacheDict',
 '_classTupleCacheDict',
 '_len',
 '_matchingElements',
 '_newBaseStream',
 '_reprInternal',
 'activeElementList',
 'activeInformation',
 'addFilter',
 'childRecursiveIterator',
 'classSet',
 'classes',
 'cleanup',
 'cleanupOnStop',
 'clone',
 'currentHierarchyOffset',
 'elementsLength',
 'filters',
 'first',
 'getElementById',
 'getElementsByClass',
 'getElementsByGroup',
 'getElementsByOffset',
 'getEleme

In [61]:
for el in p.score.recurse().notesAndRests:
    print(el.offset, el.lyric)

0.0 C#m(I)
1.0 None
1.5 None
2.0 IV
3.0 None
3.5 I
0.0 GerVI
1.0 V+
2.0 None
0.0 I
1.0 None
1.5 None
2.0 IV
3.0 None
3.5 I
0.0 GerVI
1.0 V+
2.0 None
0.0 I
2.0 II
0.0 None
2.0 None
2.125 None
2.25 None
2.375 None
2.5 None
2.625 None
2.75 None
2.8125 None
2.875 None
2.9375 None
3.0 None
3.0625 None
3.125 None
3.1875 None
3.25 None
3.3125 None
3.375 None
3.4375 None
3.5 None
3.5625 None
3.625 None
3.6875 None
3.75 None
91/24 None
23/6 None
3.875 None
3.9375 None
0.0 I
2.0 None
0.0 I+
2.0 None
2.75 None
3.0 None
10/3 None
11/3 None
0.0 II
2.0 None
0.0 V+
2.0 None
0.0 I
2.0 IV
3.0 None
0.0 None
0.0 None
0.0 None
3.0 None
3.5 None
0.0 V+
0.125 None
0.25 None
0.375 None
0.5 None
0.625 None
0.75 None
0.875 None
1.0 None
1.125 None
1.25 None
1.375 None
1.5 None
1.625 None
1.75 None
1.8125 None
1.875 None
1.9375 None
2.0 None
2.0625 None
2.125 None
2.1875 None
2.25 None
2.3125 None
2.375 None
2.4375 None
2.5 None
2.5625 None
2.625 None
2.6875 None
2.75 None
2.8125 None
2.875 None
2.9375 None
3.0